<a href="https://colab.research.google.com/github/rmonterof/Python/blob/main/1_2_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Índice

1. Series:
    * pd.Series()
    * value, index, sum()
2. DataFrames
    * 2.1 Atributos: index, columns, values, sum()
    * 2.2 Crea Dataframes:
        * listas
        * diccionarios
        * csv
        * Excel
        * json
    * 2.3 Renombrado de columnas e índices
        * rename()
        * set_index()
        * reset_index()
    * 2.4 Tipos de datos
    * 2.5 Tipo Category
    * 2.6 Indexación y selección de datos
        * keys(), items()
        * loc
        * iloc
        * Métodos para la selección de columnas: filter()
    * 2.7 Operaciones: se pueden aplicar las operaciones comentadas en Numpy
3. Limpieza de datos nulos
    * 3.1 Detección de nulos en base al tipo
    * 3.2 dropna()
    * 3.3 fillna()
    * 3.4 Contar el número de nulos en un DataFrame
4. Combinación de DataFrames
    * 4.1 Concatenación
    * 4.2 Joins: merge() keyword how
5. Agregaciones
    * 5.1 GroupBy: groupby()
    * 5.2 apply()
    

    
    
        
    


# Versiones instaladas

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
print ("Version NumPy:", np.__version__)
print ("Versión Pandas: ", pd.__version__)

Version NumPy: 1.18.5
Versión Pandas:  1.2.1


In [ ]:
import inspect
from IPython.display import display, HTML

In [ ]:
def show_dfs(*dfs, caption=""):
    '''
    Funcion de ayuda para mostrar varios dataframes en la
    misma fila.

    dfs: varios DataFrames
    '''
    res = """<table border="4" class="dataframe">
             <caption style="font-size:1.2em;">
                 <strong>{0}</strong>
             </caption>""".format(caption)
    res += '<tr>'
    for t in dfs:
        res += '<td style="border: 1px solid black; vertical-align:top;">'+t.to_html()+'</td>'
    res += '</tr></table>'
    display(HTML(res))

# Series

In [ ]:
serie = pd.Series([10,20,30])
serie

0    10
1    20
2    30
dtype: int64

In [ ]:
serie.values

array([10, 20, 30])

In [ ]:
serie.index

RangeIndex(start=0, stop=3, step=1)

Las series funcionan como los arrays de NumPy

In [ ]:
serie[1]

20

In [ ]:
serie[:2]

0    10
1    20
dtype: int64

In [ ]:
serie.sum()

60

Se puede indicar de forma explícita el índice a utilizar

In [ ]:
pd.Series ([5,3,2],index=[100,200,300])

100    5
200    3
300    2
dtype: int64

Incluso en forma de diccionario de indice y valor

In [ ]:
pd.Series ({100:'a', 200:'b', 300:'c'})

100    a
200    b
300    c
dtype: object

Pero siempre prevalece el parámetro index, en el siguiente caso se crea la Serie sólo con los elementos contenidos en el parámetro index

In [ ]:
pd.Series ({100:'a', 200:'b', 300:'c'}, index=[100,300])

100    a
300    c
dtype: object

# DataFrames

In [ ]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
area_dict = {'California': 423967,
             'Texas': 695662,
             'New York': 141297,
             'Florida': 170312,
             'Illinois': 149995}
area = pd.Series(area_dict)
estados = pd.DataFrame({'area':area, 'poblacion':population})
estados

,area,poblacion
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


Como en la clase `Series` tenemos acceso a los índices

In [ ]:
estados.index

Index(['California', 'Texas', 'New York', 'Florida', 'Illinois'], dtype='object')

A los nombres de las columnas

In [ ]:
estados.columns

Index(['area', 'poblacion'], dtype='object')

Y a los valores

In [ ]:
estados.values

array([[  423967, 38332521],
       [  695662, 26448193],
       [  141297, 19651127],
       [  170312, 19552860],
       [  149995, 12882135]])

In [ ]:
estados.sum() #Suma los valores de cada columna

area           1581233
poblacion    116866836
dtype: int64

## Creando DataFrames

### Desde colecciones en memoria

In [ ]:
data = [['Tom', 10], ['Nick', 15], ['Juli', 14]]
cols = ['Name', 'Age']
index = ['fila primera', 'fila segunda', 'fila tercera']
df = pd.DataFrame(data, columns=cols)
df

,Name,Age
0,Tom,10
1,Nick,15
2,Juli,14


In [ ]:
df = pd.DataFrame(data,index=['f1','f2','f3'])
df

,0,1
f1,Tom,10
f2,Nick,15
f3,Juli,14


In [ ]:
data_cols = {'Name':['Tom', 'nick', 'krish', 'jack'],
             'Age':[20, 21, 19, 18]}
df = pd.DataFrame(data)
df

,0,1
0,Tom,10
1,Nick,15
2,Juli,14


Con **alineación de índices**. La primera fila no tiene valores para la columna 'a'

In [ ]:
data = [{'b': 2,  'c':3},
        {'a': 10, 'b': 20, 'c': 30}]
df = pd.DataFrame(data)
df

,b,c,a
0,2,3,NaN
1,20,30,10.0


Incluso podemos elegir que nombres de índices se utilizan

In [ ]:
data = [{'b': 2, 'c':3}, {'a': 10, 'b': 20, 'c': 30}]
df = pd.DataFrame(data, index =['first', 'second'])
df

,b,c,a
first,2,3,NaN
second,20,30,10.0


### Desde ficheros

Podemos crear el DataFrame a partir de un fichero csv, los tipos de las columnas son inferidos automáticamente

In [ ]:
df=pd.read_csv ("./data/sales_info.csv")
df.head(2)

,Company,Person,Sales
0,GOOG,Sam,200
1,GOOG,Charlie,120


In [ ]:
df.dtypes

Company    object
Person     object
Sales       int64
dtype: object

Podemos especificar el tipo que queremos para las columnas del fichero, sin necesidad de explicitar todas.

In [ ]:
dfcsv=pd.read_csv ("./data/sales_info.csv", dtype={'Sales': 'float'})
dfcsv.dtypes

Company     object
Person      object
Sales      float64
dtype: object

In [ ]:
dfexcel=pd.read_excel("data/Employee.xlsx", nrows=2)
dfexcel

,first_name,last_name,company_name,address,city,county,postal,phone,email,web
0,Aleshia,Tomkiewicz,Alan D Rosenburg Cpa Pc,14 Taylor St,St. Stephens Ward,Kent,CT2 7PP,01944-369967,atomkiewicz@hotmail.com,http://www.alandrosenburgcpapc.co.uk
1,Evan,Zigomalas,Cap Gemini America,5 Binney St,Abbey Ward,Buckinghamshire,HP11 2AX,01714-737668,evan.zigomalas@gmail.com,http://www.capgeminiamerica.co.uk


Desde un fichero json

In [ ]:
with open ('./data/demo.json') as file:
    print (file.read())

[
{"value": "New", "onclick": "CreateNewDoc()"},
{"value": "Open", "onclick": "OpenDoc()"},
{"value": "Close", "onclick": "CloseDoc()"}
]



In [ ]:
dfjson = pd.read_json ("./data/demo.json")
dfjson

,value,onclick
0,New,CreateNewDoc()
1,Open,OpenDoc()
2,Close,CloseDoc()


## Renombrando columnas e índices

Una vez creado el dataframe podemos modificar sus columnas modificando directamente la propiedad `columns` del dataframe

In [ ]:
dfjson.columns = ['Función', 'evento']
dfjson

,Función,evento
0,New,CreateNewDoc()
1,Open,OpenDoc()
2,Close,CloseDoc()


Hay que tener en cuenta que la modificación de la propiedad `columns` se hace sobre el propio dataframe, no genera un nuevo dataframe.  

También podemos cambiar los nombres de forma más selectiva a través del método `rename`. En este caso se genera un nuevo dataframe, no se modifica sobre el exsistente.

In [ ]:
dfjson.rename(columns={"evento":"evento_onclick"})

,Función,evento_onclick
0,New,CreateNewDoc()
1,Open,OpenDoc()
2,Close,CloseDoc()


Para modificar los índices de un dataframe podemos utilizar la asignación directa a la propiedad `index` del dataframe de un vector con los nuevos valores.

In [ ]:
dfjson.index=range(4,7)
dfjson

,Función,evento
4,New,CreateNewDoc()
5,Open,OpenDoc()
6,Close,CloseDoc()


También podemos utilizar los métodos `set_index` y `reset_index` para convertir columnas en índices o convertir índices en columnas

In [ ]:
dfjson2 = dfjson.set_index('Función')
dfjson3 = dfjson2.reset_index()
show_dfs (dfjson, dfjson2, dfjson3)

# Tipos

A continuación creamos un dataframe cuyas columnas tienen algunos tipos utilizados habitualmente, hay que tener en cuenta que `int*, Int* y float*` pueden tener muchos tipos dependiendo de la precisión.
También es conveniente fijarse en que los tipos `int` no permiten nulos, si los necesitamos tenemos que recurrir a los equivalentes `Int` (*empiezan por mayúscula*).

In [ ]:
nums=range(1,4)
alf=list('abc')
logic = [True,False]
df = pd.DataFrame({
    'c_int64'     : pd.Series(nums, dtype="int64"),
    'c_Int64'     : pd.Series(nums, dtype="Int64"),
    'c_str'       : pd.Series(alf, dtype='str_'),
    'c_cat'       : pd.Series(alf, dtype='category'),
    'c_boolean'   : pd.Series(logic, dtype='boolean'),
    'c_bool'      : pd.Series(logic, dtype='bool'),
    'c_float'     : pd.Series(nums, dtype='float'),
    'c_datetime64': pd.Series(nums, dtype="datetime64[ns]"),
})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   c_int64       3 non-null      int64         
 1   c_Int64       3 non-null      Int64         
 2   c_str         3 non-null      object        
 3   c_cat         3 non-null      category      
 4   c_boolean     2 non-null      boolean       
 5   c_bool        2 non-null      object        
 6   c_float       3 non-null      float64       
 7   c_datetime64  3 non-null      datetime64[ns]
dtypes: Int64(1), boolean(1), category(1), datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 416.0+ bytes


In [ ]:
df

,c_int64,c_Int64,c_str,c_cat,c_boolean,c_bool,c_float,c_datetime64
0,1,1,a,a,True,True,1.0,1970-01-01 00:00:00.000000001
1,2,2,b,b,False,False,2.0,1970-01-01 00:00:00.000000002
2,3,3,c,c,<NA>,NaN,3.0,1970-01-01 00:00:00.000000003


## Tipo: Category


[Differences to R’s factor](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html#differences-to-r-s-factor)

The following differences to R’s factor functions can be observed:

* R’s `levels` are named `categories`.
* R’s `levels` are always of type string, while `categories` in pandas can be of any dtype.
* It’s not possible to specify `labels` at creation time. Use `s.cat.rename_categories(new_labels)` afterwards.
* In contrast to R’s `factor` function, using categorical data as the sole input to create a new categorical series will not remove unused categories but create a new categorical series which is equal to the passed in one!
* R allows for missing values to be included in its `levels` (pandas’ `categories`). pandas does not allow `NaN` categories, but missing values can still be in the `values`.

Es un tipo de dato para almacenar datos categóricos, tiene las propiedades `categories` y `ordered`.  
Para acceder a estas propiedades en un DataFrame se hace a través del método `cat` de la clase `pandas.Series`

In [ ]:
print ('Tipo columna: ', type(df.c_cat))
print ('Tipo cat: ', type(df.c_cat.cat))
print ('Categorías: ', df.c_cat.cat.categories)
print ('Índice a la categoría', df.c_cat.values.codes)
print ('Ordered: ', df.c_cat.cat.ordered)
df.c_cat

Tipo columna:  <class 'pandas.core.series.Series'>
Tipo cat:  <class 'pandas.core.arrays.categorical.CategoricalAccessor'>
Categorías:  Index(['a', 'b', 'c'], dtype='object')
Índice a la categoría [0 1 2]
Ordered:  False


0    a
1    b
2    c
Name: c_cat, dtype: category
Categories (3, object): ['a', 'b', 'c']

In [ ]:
df.c_cat.value_counts()

a    1
b    1
c    1
Name: c_cat, dtype: int64

### Podemos renombrar las categorías

In [ ]:
df.c_cat.cat.categories = ['Grupo a', 'Grupo b', 'Grupo c']
df.c_cat

0    Grupo a
1    Grupo b
2    Grupo c
Name: c_cat, dtype: category
Categories (3, object): ['Grupo a', 'Grupo b', 'Grupo c']

La mayoría de los métodos de la clase generan un nuevo objeto, pero **renombrar** las categorias **se hace sobre el propio objeto (in place)**  

Cuando renombramos las categorías las nuevas categorías sustituyen a las antiguas según el orden en el que aparecen en el array *Categories*.

In [ ]:
c = pd.Categorical (list('dcba'),ordered=False)
c

['d', 'c', 'b', 'a']
Categories (4, object): ['a', 'b', 'c', 'd']

In [ ]:
c.categories=list('1234')
c

['4', '3', '2', '1']
Categories (4, object): ['1', '2', '3', '4']

In [ ]:
v_cat = pd.Categorical ([3,3,4,5,2,1,6,6], ordered=True)
v_cat

[3, 3, 4, 5, 2, 1, 6, 6]
Categories (6, int64): [1 < 2 < 3 < 4 < 5 < 6]

In [ ]:
v_cat.categories = [f'C{i}' for i in range (1,7)]
v_cat

['C3', 'C3', 'C4', 'C5', 'C2', 'C1', 'C6', 'C6']
Categories (6, object): ['C1' < 'C2' < 'C3' < 'C4' < 'C5' < 'C6']

### Categorías: Añadir, eliminar, ordenar, ...

Cuando añadimos o eliminamos categorías se genera un nuevo objeto

In [ ]:
v_cat.add_categories(['Cara 7'])

['C3', 'C3', 'C4', 'C5', 'C2', 'C1', 'C6', 'C6']
Categories (7, object): ['C1' < 'C2' < 'C3' < 'C4' < 'C5' < 'C6' < 'Cara 7']

In [ ]:
v_cat.add_categories(['Cara 7']).value_counts()  #Cuenta los que no tienen valores, como 'Cara 7'

C1        1
C2        1
C3        2
C4        1
C5        1
C6        2
Cara 7    0
dtype: int64

In [ ]:
v_cat.remove_categories(['Cara 1'])

['Cara 3', 'Cara 3', 'Cara 4', 'Cara 5', 'Cara 5', 'Cara 2', NaN, NaN, 'Cara 6', 'Cara 6']
Categories (5, object): ['Cara 2' < 'Cara 3' < 'Cara 4' < 'Cara 5' < 'Cara 6']

In [ ]:
letras = pd.Categorical(list('aaabb'),categories=list('abc'))
letras

['a', 'a', 'a', 'b', 'b']
Categories (3, object): ['a', 'b', 'c']

In [ ]:
letras.remove_unused_categories() #elimina 'c'

['a', 'a', 'a', 'b', 'b']
Categories (2, object): ['a', 'b']

Si el objeto de tipo `category` está ordenado tienen sentido los métodos `.min()/.max()`

In [ ]:
nums = pd.Categorical(np.random.choice(range(1,11),10,replace=False),
                      categories=range(1,11),
                      ordered=True)
nums

[9, 3, 4, 8, 6, 5, 10, 7, 2, 1]
Categories (10, int64): [1 < 2 < 3 < 4 ... 7 < 8 < 9 < 10]

In [ ]:
print ('El valor máximo: ', nums.max())
print ('Está en la posición: ', np.where (nums==nums.max()))

El valor máximo:  10
Está en la posición:  (array([6]),)


Se puede convertir una `category` en `ordered`.  
Hay que tener en cuenta que por defecto las categorías las ordena de menor a mayor, y al convertirla en un `category` ordenado se mantiene ese orden porque el `.as_ordered()` convierte a categoría ordenada, pero manteniendo el orden previo que tenían las categorías

In [ ]:
nums = pd.Categorical(np.random.choice(range(1,6),5,replace=False))
nums

[5, 4, 2, 1, 3]
Categories (5, int64): [1, 2, 3, 4, 5]

In [ ]:
nums.as_ordered()

[5, 4, 2, 1, 3]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

In [ ]:
nums = pd.Categorical(np.random.choice(range(1,6),5,replace=False),
                     categories=[5,4,3,2,1])
nums.as_ordered()

[4, 1, 2, 5, 3]
Categories (5, int64): [5 < 4 < 3 < 2 < 1]

In [ ]:
nums.as_ordered()

[4, 1, 2, 5, 3]
Categories (5, int64): [5 < 4 < 3 < 2 < 1]

Las variables categóricas no están soportadas por la clase `numpy.ndarray`

In [ ]:
nums = pd.Series(range(1,5),dtype="category")
try:
    np.sum(nums)
except TypeError as e:
    print ("Error: ", str(e))

Error:  'Categorical' does not implement reduction 'sum'


In [ ]:
np.sum(nums.astype('int'))

10

Ocupan menos memoria que las variables habituales, siempre que el número de categorías no sea tan alto como el número de elementos

In [ ]:
s = pd.Series(["Categoría 1", "Categoría 2"] * 1000)
print ('Habitual: ', s.nbytes, 'bytes')
print ('Categórica: ', s.astype('category').nbytes, 'bytes')

Habitual:  16000 bytes
Categórica:  2016 bytes


# Indexación y Selección de datos

In [ ]:
pop = [38332521,26448193,19651127,19552860, 12882135]
area = [423967, 695662, 141297, 170312, 149995]
state = ['CA', 'TX', 'NY', 'FL', 'IL']
cols = ['Poblacion', 'Area', 'Estado']
cities = ['Los Ángeles', 'Houston', 'New York','Miami', 'Chicago']
ciudades = pd.DataFrame({'Poblacion':pop,'Area':area,'Estado':state},
                        index=cities)
ciudades

,Poblacion,Area,Estado
Los Ángeles,38332521,423967,CA
Houston,26448193,695662,TX
New York,19651127,141297,NY
Miami,19552860,170312,FL
Chicago,12882135,149995,IL


Como el DataFrame está formados por columnas de la clase `Series` y esta clase se indexa como la clase `numpy.ndarray` podemos acceder a los valores de una serie por el nombre del índice. Se muestra el acceso al valor de *Texas* de la columna *area*

In [ ]:
ciudades['Area']['Houston']

695662

Podemos verificar si un valor está en el índice del DataFrame

In [ ]:
'Miami' in ciudades['Area'] # igual que 'Miami' in ciudades.index

True

## Accediendo a las columnas(`Series`) - 1 dimensión

Podemos utilizar las funciones de diccionario para acceder a los elementos de una columna

In [ ]:
ciudades['Area'].keys()

Index(['Los Ángeles', 'Houston', 'New York', 'Miami', 'Chicago'], dtype='object')

In [ ]:
list(ciudades['Area'].items())

[('Los Ángeles', 423967),
 ('Houston', 695662),
 ('New York', 141297),
 ('Miami', 170312),
 ('Chicago', 149995)]

**Mediante posición**

In [ ]:
ciudades.Area[0]

423967

**Mediante slicing**, rangos de índices

In [ ]:
ciudades['Area']['Houston':'Miami']

Houston     695662
New York    141297
Miami       170312
Name: Area, dtype: int64

In [ ]:
a= 'Area'
ciudades[a][1:4]

Houston     695662
New York    141297
Miami       170312
Name: Area, dtype: int64

**Masking**, array de booleanos

In [ ]:
masking=(ciudades[a]>150_000) & (ciudades[a]<500_000)
masking

Los Ángeles     True
Houston        False
New York       False
Miami           True
Chicago        False
Name: Area, dtype: bool

In [ ]:
ciudades.Area[masking]

Los Ángeles    423967
Miami          170312
Name: Area, dtype: int64

**Fancy**, array con los índices a recuperar

In [ ]:
ciudades.Area[['Miami','New York']]

Miami       170312
New York    141297
Name: Area, dtype: int64

## Accediendo al DataFrame - 2 dimensiones

Un DataFrame en un array de 2 dimensiones *vitaminado*

In [ ]:
ciudades.values

array([[38332521, 423967, 'CA'],
       [26448193, 695662, 'TX'],
       [19651127, 141297, 'NY'],
       [19552860, 170312, 'FL'],
       [12882135, 149995, 'IL']], dtype=object)

Al ser un array de 2 dimensiones, podemos, por ejemplo, calcula el *transpuesto*

In [ ]:
ciudades.T

,Los Ángeles,Houston,New York,Miami,Chicago
Poblacion,38332521,26448193,19651127,19552860,12882135
Area,423967,695662,141297,170312,149995
Estado,CA,TX,NY,FL,IL


Para el acceso a una *porción* de DataFrame utilizaremos los comando `loc` e `iloc`  
* `loc`: Acceso mediante los nombres de índices de filas y columnas
* `iloc`: Acceso mediante la posiciones de filas y columnas

Cuando accedemos a los datos de un DataFrame tenemos que indicar el subconjunto de datos para cada dimensión. En la siguiente instrucciones obtenemos el subconjunto formado por la intersección de la fila con índice *Miami* y la columna de nombre *Area*.  
Los subconjuntos pueden involucrar a varias filas y/o columnas

In [ ]:
ciudades.loc['Miami','Area']

170312

In [ ]:
ciudades.iloc[3,1]

170312

Como veíamos en el acceso a las columnas, aquí también podemos utilizar los modos de acceso posicionales, slicing, masking y fancy, tanto a la dimensión de las filas (*axis o*), como a la dimensión de las columnas (*axis 1*)

In [ ]:
ciudades.iloc [1,:]

Poblacion    26448193
Area           695662
Estado             TX
Name: Houston, dtype: object

In [ ]:
ciudades.loc ['Houston',:]

Poblacion    26448193
Area           695662
Estado             TX
Name: Houston, dtype: object

In [ ]:
ciudades.iloc[:2,1]

Los Ángeles    423967
Houston        695662
Name: Area, dtype: int64

In [ ]:
ciudades.loc[:'Houston','Area']

Los Ángeles    423967
Houston        695662
Name: Area, dtype: int64

Recordemos que cualquier array de booleanos se interpreta como una máscara, de forma que solo se seleccionan las filas (o columnas si se aplica en esa dimensión), cuya posición coincide con valores `True`.  
Por eso cualquier condición booleana genera un filtro de las filas que la cumplen

In [ ]:
mask=[True,False,True,False,True]
ciudades.loc[mask,'Estado']

Los Ángeles    CA
New York       NY
Chicago        IL
Name: Estado, dtype: object

In [ ]:
ciudades.iloc[mask,2]

Los Ángeles    CA
New York       NY
Chicago        IL
Name: Estado, dtype: object

In [ ]:
ciudades.iloc[[0,2,4],2]

Los Ángeles    CA
New York       NY
Chicago        IL
Name: Estado, dtype: object

Las mismas funciones que se utilizan para acceder a los elementos nos van a permitir modificar los valores de los elementos seleccionados

In [ ]:
ciudades['colTest'] = 1
ciudades

,Poblacion,Area,Estado,colTest
Los Ángeles,38332521,423967,CA,1
Houston,26448193,695662,TX,1
New York,19651127,141297,NY,1
Miami,19552860,170312,FL,1
Chicago,12882135,149995,IL,1


In [ ]:
ciudades.loc[['Miami','New York'],'colTest'] += 2
ciudades

,Poblacion,Area,Estado,colTest
Los Ángeles,38332521,423967,CA,1
Houston,26448193,695662,TX,1
New York,19651127,141297,NY,3
Miami,19552860,170312,FL,3
Chicago,12882135,149995,IL,1


En general en python se pueden utilizar la sintaxis de posición negativa de los índices para acceder a los últimos elementos, también es válido para la función `iloc` de los DataFrames

In [ ]:
ciudades.iloc[-2,3]==ciudades.loc['Miami','colTest']

True

##

## Métodos para selección de columnas

In [ ]:
movies=pd.read_csv("data/movie.csv",nrows=30)
print ("Dimensiones", movies.shape)
movies.dtypes.value_counts()

Dimensiones (30, 28)


float64    13
object     12
int64       3
dtype: int64

Podemos seleccionar todas las columnas numéricas de un DataFrame

In [ ]:
movies.select_dtypes(include="number").shape

(30, 16)

O seleccionar todas las columnas excepto las de uno o varios tipos

In [ ]:
movies.select_dtypes(exclude=["int64","float64"]).shape

(30, 12)

In [ ]:
movies.columns

Index(['color', 'director_name', 'num_critic_for_reviews', 'duration',
       'director_facebook_likes', 'actor_3_facebook_likes', 'actor_2_name',
       'actor_1_facebook_likes', 'gross', 'genres', 'actor_1_name',
       'movie_title', 'num_voted_users', 'cast_total_facebook_likes',
       'actor_3_name', 'facenumber_in_poster', 'plot_keywords',
       'movie_imdb_link', 'num_user_for_reviews', 'language', 'country',
       'content_rating', 'budget', 'title_year', 'actor_2_facebook_likes',
       'imdb_score', 'aspect_ratio', 'movie_facebook_likes'],
      dtype='object')

In [ ]:
movies.filter(like="actor_1").columns

Index(['actor_1_facebook_likes', 'actor_1_name'], dtype='object')

# Operaciones sobre DataFrames

Los DataFrame heredan todas las propiedades de los `numpy.ndarrays` (excepto para la clase `category`), por lo tanto modemos aplicar sobre ellos todas las funciones que se han visto para *NumPy*.  
En el caso de los DataFrame tenemos dos comportamientos añadidos:
* El resultado de una operación unaria conserva los índices y el nombre de la columna
* Cuando se realiza una operación binaria pandas previsamente realiza una alineación de índices

In [ ]:
df = pd.DataFrame({'col1':range(1,4),'col2':range(11,14)})
df

,col1,col2
0,1,11
1,2,12
2,3,13


In [ ]:
np.exp(df) #conserva índices y columnas

,col1,col2
0,2.718282,59874.141715
1,7.389056,162754.791419
2,20.085537,442413.392009


In [ ]:
s1 = pd.Series ({'a':1, 'c':3},name='serie1')
s2 = pd.Series ({'b':20, 'a':10, 'c':30},name='serie2') #No están en el mismo orden
s1+s2

a    11.0
b     NaN
c    33.0
dtype: float64

In [ ]:
rnd = np.random.RandomState(1)

In [ ]:
A = pd.DataFrame (rnd.randint(1,10,(2,2)), columns=list('ab'))
A

,a,b
0,6,9
1,6,1


In [ ]:
B = pd.DataFrame(rnd.randint(1,10,(3,3)), columns= list('cba'))
B

,c,b,a
0,1,2,8
1,7,3,5
2,6,3,5


In [ ]:
A+B

,a,b,c
0,14.0,11.0,NaN
1,11.0,4.0,NaN
2,NaN,NaN,NaN


También tenemos operadores sustitutivos de las operaciones básicas de python  

|Operador python|Método Pandas|  
|:--------------|:---------------|  
|+|add()|
|-|sub(), subtract()|
|\*|mul(), multiply()|
|/|div(), divide()|
|//|floordiv()|
|%|mod()|
|\*\*|pow()|



In [ ]:
to_fill = A.stack().mean()
to_fill

5.5

In [ ]:
A.add(B,fill_value=to_fill)

,a,b,c
0,14.0,11.0,6.5
1,11.0,4.0,12.5
2,10.5,8.5,11.5


In [ ]:
A

,a,b
0,6,9
1,6,1


In [ ]:
A.sub(A.iloc[0])

,a,b
0,0,0
1,0,-8


In [ ]:
A.sub(A.a,axis=0) #axis=0, resta por coincidencia de índice

,a,b
0,0,3
1,0,-5


# Trabajando con datos nulos

En pandas nos encontramos dos tipos de nulos, el propio de python(`None`) y el derivado de numpy (`numpy.NaN`)

El uso de `None` (es un objeto) requiere que la columna sea de tipo `object`, y los cálculos se ven penalizados

In [ ]:
%timeit np.arange(1000, dtype='object').sum()
%timeit np.arange(1000, dtype='int').sum()

30.1 µs ± 733 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
3.72 µs ± 41.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Además, en las operaciones generar errores

In [ ]:
try:
    1+None
except TypeError as e:
    print ("Error: ", str(e))

Error:  unsupported operand type(s) for +: 'int' and 'NoneType'


In [ ]:
1+np.nan

nan

In [ ]:
df = pd.DataFrame({
    'c_int64'     : pd.Series([1,    2,    3,     4],      dtype="int64"),  #No permite nulos
    'c_Int64'     : pd.Series([1,    None, 3,     np.nan], dtype="Int64"),
    'c_str'       : pd.Series(['a',  None, 'c',   np.nan], dtype='str_'),
    'c_cat'       : pd.Series(['a',  None, 'c',   np.nan], dtype='category'),
    'c_boolean'   : pd.Series([True, None, False, np.nan], dtype='boolean'),
    'c_bool'      : pd.Series([True, None, False, np.nan], dtype='bool'),
    'c_float'     : pd.Series([1,    None, 3,     np.nan], dtype='float'),
    'c_datetime64': pd.Series([1,    None, 3,     np.nan], dtype="datetime64[ns]"),
})
df

,c_int64,c_Int64,c_str,c_cat,c_boolean,c_bool,c_float,c_datetime64
0,1,1,a,a,True,True,1.0,1970-01-01 00:00:00.000000001
1,2,<NA>,None,NaN,<NA>,False,NaN,NaT
2,3,3,c,c,False,False,3.0,1970-01-01 00:00:00.000000003
3,4,<NA>,NaN,NaN,<NA>,True,NaN,NaT


Nos devuelve una matriz booleana con True en las posiciones de los nulos

In [ ]:
df.isnull() # igual que df.isna()

,c_int64,c_Int64,c_str,c_cat,c_boolean,c_bool,c_float,c_datetime64
0,False,False,False,False,False,False,False,False
1,False,True,True,True,True,False,True,True
2,False,False,False,False,False,False,False,False
3,False,True,True,True,True,False,True,True


La operación contraria, `True` cuando **no** es nulo

In [ ]:
df.notnull()

,c_int64,c_Int64,c_str,c_cat,c_boolean,c_bool,c_float,c_datetime64
0,True,True,True,True,True,True,True,True
1,True,False,False,False,False,True,False,False
2,True,True,True,True,True,True,True,True
3,True,False,False,False,False,True,False,False


In [ ]:
df.dropna() #elimina filas que tienen nulos

,c_int64,c_Int64,c_str,c_cat,c_boolean,c_bool,c_float,c_datetime64
0,1,1,a,a,True,True,1.0,1970-01-01 00:00:00.000000001
2,3,3,c,c,False,False,3.0,1970-01-01 00:00:00.000000003


In [ ]:
df.dropna(axis=1) # para eliminar columnas

,c_int64,c_bool
0,1,True
1,2,False
2,3,False
3,4,True


En el caso del `.dropna()` podemos tener más control eliminando una fila o columan cuando tiene todos los elementos nulos o controlando cuantos elementos no nulos deben tener las filas/columnas que no se borran.

In [ ]:
A = pd.DataFrame(np.random.randint(1,11,(5,4)),columns=list('abcd'))
A.iloc[0,:2]=np.nan
A.iloc[1,:]=np.nan
A.iloc[2,0]=np.nan
A

,a,b,c,d
0,NaN,NaN,6.0,3.0
1,NaN,NaN,NaN,NaN
2,NaN,8.0,7.0,9.0
3,10.0,1.0,9.0,4.0
4,7.0,7.0,5.0,4.0


Para eliminar una fila cuando todos los elementos son nulos

In [ ]:
A.dropna(axis='index',how="all") #igual que axis=0

,a,b,c,d
0,NaN,NaN,6.0,3.0
2,NaN,8.0,7.0,9.0
3,10.0,1.0,9.0,4.0
4,7.0,7.0,5.0,4.0


In [ ]:
A.dropna(axis=1, thresh=3) #Al menos 3 elementos no NaN

,b,c,d
0,NaN,6.0,3.0
1,NaN,NaN,NaN
2,8.0,7.0,9.0
3,1.0,9.0,4.0
4,7.0,5.0,4.0


Tambien podemos sustituir el valor nulo por otro valor, en este caso se pueden elegir tres opciones:  
* Por un valor fijo, pero todas las columnas deben tener tipos compatibles con el valor
* Por el valor no nulo previo que encuentre
* Por el valor siguiente no nulo que encuentre

Toda la fila 1 contiene nulos, en el primer caso se reemplazan por 99, en el segundo se reemplazan solo para la columna 2 y 3, que son los que tienen valores previos, y en el tercer caso se reemplazan para todas las columnas con los valores posteriores.

In [ ]:
A.fillna(99)

,a,b,c,d
0,99.0,99.0,6.0,3.0
1,99.0,99.0,99.0,99.0
2,99.0,8.0,7.0,9.0
3,10.0,1.0,9.0,4.0
4,7.0,7.0,5.0,4.0


In [ ]:
A.fillna(method='ffill')

,a,b,c,d
0,NaN,NaN,6.0,3.0
1,NaN,NaN,6.0,3.0
2,NaN,8.0,7.0,9.0
3,10.0,1.0,9.0,4.0
4,7.0,7.0,5.0,4.0


In [ ]:
A.fillna(method='bfill')

,a,b,c,d
0,10.0,8.0,6.0,3.0
1,10.0,8.0,7.0,9.0
2,10.0,8.0,7.0,9.0
3,10.0,1.0,9.0,4.0
4,7.0,7.0,5.0,4.0


## ¿Cuántos nulos hay en el DataFrame?

Esta función presenta información sobre los nulos que hay en un dataset por columnas.  
> La ejecución de la siguiente celda genera un error controlado, ya que dentro de la función se comprueba que no se aplique a DataFrames de más de 10 elementos en una de sus dimensiones

In [ ]:
# Se podría utilizar ValueError como excepción
# Se prefiere definir excepción propia y así
# se tiene un ejemplo de como hacerlo
class ShapeError (Exception):
    pass

def informacion_nulos(df, axis=1):
    if (max(df.shape)>10):
        raise ShapeError(f'El tamaño del DataFrame {df.shape} es muy grande. Longitud máxima en cualquier dimensión 10')
    if axis==0:
        df = df.T
    nulos = pd.DataFrame ({'datos': list(df.T.values),
                           'cant':df.isnull().sum(),
                           'alguno':df.isnull().any(),
                           'todos':df.isnull().all(),
                          })
    return nulos

#Para prueba de DataFrame grandes
try:
    informacion_nulos(pd.DataFrame(np.ones((5,11))))
except ShapeError as e:
    print ("Error: ",e)

Error:  El tamaño del DataFrame (5, 11) es muy grande. Longitud máxima en cualquier dimensión 10


In [ ]:
informacion_nulos(A)   #por columnas

,datos,cant,alguno,todos
a,"[nan, nan, nan, 10.0, 7.0]",3,True,False
b,"[nan, nan, 8.0, 1.0, 7.0]",2,True,False
c,"[6.0, nan, 7.0, 9.0, 5.0]",1,True,False
d,"[3.0, nan, 9.0, 4.0, 4.0]",1,True,False


In [ ]:
informacion_nulos(A.T) #por filas

,datos,cant,alguno,todos
0,"[nan, nan, 6.0, 3.0]",2,True,False
1,"[nan, nan, nan, nan]",4,True,True
2,"[nan, 8.0, 7.0, 9.0]",1,True,False
3,"[10.0, 1.0, 9.0, 4.0]",0,False,False
4,"[7.0, 7.0, 5.0, 4.0]",0,False,False


In [ ]:
print ("Cantidad nulos en el DataFrame: ", A.isnull().sum().sum())
print ("Algún nulo en el DataFrame: ", A.isnull().any().any())

Cantidad nulos en el DataFrame:  7
Algún nulo en el DataFrame:  True
